In [4]:
import csv
import json
import random
from openai import OpenAI
from time import sleep
from dotenv import load_dotenv
import os

load_dotenv()  

True

# Converting raw data to training and validation datasets.

**Creating the training data. each entry in the json looks like:**

{"messages": [{"role": "system", "content": "Identify body parts and main target muscles from workouts."}, <br/>
{"role": "user", "content": "Lying Crunch (straight legs)  using Body weight"}, <br/>
{"role": "assistant", "content": "Targets Waist:  but also hits "}]} <br/>


In [15]:
def convert_csv_to_training_format(rows, output_file):
    system_message = {
        "role": "system",
        "content": "Identify body parts and main target muscles from workouts."
    }

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for row in rows:
            exercise = row[1].replace("(female)","").replace("(male)","")
            equipment = row[4]
            general_bodypart = row[3]
            target_bodypart = row[6]
            also_hits = row[7]
            training_example = {
                "messages": [
                    system_message,
                    {"role": "user", "content": f'{exercise} using {equipment}'},
                    {"role": "assistant", "content": f"Targets {general_bodypart}: {target_bodypart} but also hits {also_hits}"}
                ]
            }
            outfile.write(json.dumps(training_example) + '\n')

creating training & validation datasets, using 80/20 train/val split

In [8]:
with open("unprocessed_dataset.csv", 'r', encoding='utf-8') as csvfile:
    reader = list(csv.reader(csvfile))
    header = reader[0]
    rows = reader[1:]  # skip header

random.shuffle(rows)  # shuffle for randomness

split_idx = int(0.8 * len(rows))  # 80/20 train/val split
train_rows = rows[:split_idx]
val_rows = rows[split_idx:]


convert_csv_to_training_format(train_rows, "training_data.jsonl")
convert_csv_to_training_format(val_rows, "validation_data.jsonl")


# Starting the fine-tuning job

Uploading the training and validation files to openai

In [ ]:

client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))
def upload_training_file(file_path):
    """Upload training file to OpenAI"""
    with open(file_path, "rb") as file:
        response = client.files.create(
            file=file,
            purpose="fine-tune"
        )
        return response.id

training_file_id = upload_training_file("training_data.jsonl")
validation_file_id = upload_training_file("validation_data.jsonl")


Create the fine-tuning job using gpt-4o.

In [18]:
def create_fine_tuning_job(training_file_id, validation_file_id, model="gpt-4o-mini-2024-07-18"):
    """Create a fine-tuning job"""
    response = client.fine_tuning.jobs.create(
        training_file=training_file_id,
        validation_file=validation_file_id,
        model=model
    )
    return response.id

job_id = create_fine_tuning_job(training_file_id, validation_file_id)

In [ ]:
def monitor_job(job_id):
    """Monitor fine-tuning job progress"""
    while True:
        job = client.fine_tuning.jobs.retrieve(job_id)
        print(f"Status: {job.status}")

        if job.status in ["succeeded", "failed"]:
            return job

        # List latest events
        events = client.fine_tuning.jobs.list_events(
            fine_tuning_job_id=job_id,
            limit=5
        )
        for event in events.data:
            print(f"Event: {event.message}")

        sleep(30)  # check every 30 seconds

job = monitor_job(job_id)
if job.status == "succeeded":
    fine_tuned_model = job.fine_tuned_model
    print(f"Fine-tuned model ID: {fine_tuned_model}")
else:
    print("Fine-tuning failed.")

Status: validating_files
Event: Validating training file: file-Ss8NuGoyTij1sDUZNq3LDs and validation file: file-1XeujZHDyQV5aAsdCWph93
Event: Created fine-tuning job: ftjob-NRCUdWGuejuW8scwDWBV01mT
Status: validating_files
Event: Validating training file: file-Ss8NuGoyTij1sDUZNq3LDs and validation file: file-1XeujZHDyQV5aAsdCWph93
Event: Created fine-tuning job: ftjob-NRCUdWGuejuW8scwDWBV01mT
Status: validating_files
Event: Validating training file: file-Ss8NuGoyTij1sDUZNq3LDs and validation file: file-1XeujZHDyQV5aAsdCWph93
Event: Created fine-tuning job: ftjob-NRCUdWGuejuW8scwDWBV01mT
Status: validating_files
Event: Validating training file: file-Ss8NuGoyTij1sDUZNq3LDs and validation file: file-1XeujZHDyQV5aAsdCWph93
Event: Created fine-tuning job: ftjob-NRCUdWGuejuW8scwDWBV01mT
Status: validating_files
Event: Validating training file: file-Ss8NuGoyTij1sDUZNq3LDs and validation file: file-1XeujZHDyQV5aAsdCWph93
Event: Created fine-tuning job: ftjob-NRCUdWGuejuW8scwDWBV01mT
Status: va

# Testing the model

In [ ]:
def test_model(model_id, test_input):
    """Test the fine-tuned model"""
    completion = client.chat.completions.create(
        model=model_id,
        messages=[
            {
                "role": "system",
                "content": "What does this exercise target?"
            },
            {"role": "user", "content": test_input}
        ]
    )
    return completion.choices[0].message

In [24]:

test_input = """
My pull workout consists of inverted rows, front-lever raises, chinups, and swimmers.

Are there other exercises i can mix in to target more areas of my back?
"""
result = test_model(fine_tuned_model, test_input)

print(result.content)

Yes, there are several exercises you can incorporate into your pull workout to target different areas of your back. Here are some options:

1. **Pull-Ups**: A staple back exercise that hits your lats and upper back but offers a different grip than chin-ups.
  
2. **Barbell Rows**: This can include bent-over rows, pendlay rows, or Yates rows, all of which target the mid and upper back, as well as the lats.
   
3. **Dumbbell Rows**: Single-arm dumbbell rows or chest-supported rows can help isolate various areas of your back more effectively than barbells.
   
4. **Lat Pulldowns**: A great alternative to pull-ups that allows you to adjust weight and focus on your lats.
   
5. **Face Pulls**: Targets the rear deltoids and upper back muscles, particularly the traps.
   
6. **T-Bar Rows**: Excellent for building thickness in the back, targeting the mid-back muscles.
   
7. **Deadlifts**: While primarily a lower-body exercise, deadlifts are exceptional for overall back development.
   
8. **S